## Recommender System Project

Recommender System Project¶
- Type of Recommender system
   
   Content Bases                  - Collaborative filtering                 -Hybrid
   
       [tags]                          [uses data of simmiler usee        Content + Collabarative
- Type of Recommender system

We'll make Content base recommender syster

## Project FLow
 -Data Collection--->Prepocessing --->model --->website ---> Deploy

In [271]:
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import warnings
warnings.filterwarnings("ignore")

In [119]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")


In [120]:
df = movies.merge(credits,on='title') #df1.merge(df2, on='column name')

In [29]:
profile = ProfileReport(df)
profile.to_file("movie_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Data Cleaning

In [122]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [123]:
df =df[['movie_id' , 'title' , 'overview' , 'genres', 'keywords' , 'cast' , 'crew']]

In [125]:
df.dropna(inplace = True)

## Data Featuring

In [128]:
df['genres'] = df['genres'].apply(convert)

In [130]:
df['keywords'] = df['keywords'].apply(convert)

In [134]:
def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter !=3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [135]:
df['cast'] = df['cast'].apply(convert3)

In [140]:
def convert4(obj):
    L = []

    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [142]:
df['crew']= df['crew'].apply(convert4)

In [144]:
df['overview'] = df['overview'].apply(lambda x:x.split())

In [147]:
df['genres']= df['genres'].apply(lambda x:[i.replace(" " ,"") for i in x])
df['keywords']= df['keywords'].apply(lambda x:[i.replace(" " ,"") for i in x])
df['cast']= df['cast'].apply(lambda x:[i.replace(" " ,"") for i in x])
df['crew']= df['crew'].apply(lambda x:[i.replace(" " ,"") for i in x])

In [149]:
df['tags'] = df['overview'] + df['genres'] + df['keywords'] + df['cast'] + df['crew']

In [151]:
df = df[['movie_id' , 'title', 'tags']]

In [155]:
df['tags']= df['tags'].apply(lambda x:" ".join(x))

In [158]:
df['tags']= df['tags'].apply(lambda x:x.lower())


## Model building

In [227]:

cv = CountVectorizer(max_features = 5000, stop_words='english')

In [228]:
vectors = cv.fit_transform(df['tags']).toarray()

In [230]:
len(cv.get_feature_names_out())

5000

In [232]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " " .join(y)
    

In [233]:
df['tags']=df['tags'].apply(stem)

In [234]:
df.shape

(4806, 3)

In [249]:
simmilarity = cosine_similarity(vectors)

In [259]:
def recommend(movie):
    movie_index = df[df['title'] == movie].index[0]
    distances = simmilarity[movie_index]
    movie_list = sorted(list(enumerate(distances)),reverse = True, key= lambda x:x[1])[1:6]
    
    for i in movie_list:
        print(df.iloc[i[0]].title)


In [266]:
pickle.dump(df.to_dict(),open('movie.pkl','wb'))

In [270]:
pickle.dump(simmilarity,open('simmilarity.pkl','wb'))